# Finetuned GPT-4o-mini

In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2


In [2]:
import time
import json
import os
from openai import OpenAI

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

In [4]:
def str_to_bool(string):
  if string == "True":
    return True
  elif string == "False":
    return False
  return False

In [5]:
test_set = []
y_test = []

with open("test_set.jsonl", "r") as file:
  for line in file:
    data = json.loads(line)
    test_set.append(data['messages'])
    y_test.append(str_to_bool(data['messages'][2]['content']))

In [6]:
api_key = "" # TODO
os.environ['OPENAI_API_KEY'] = api_key

In [7]:
client = OpenAI()
our_model = "gpt-4o-mini-2024-07-18"

## Finetuning

In [8]:
response = client.files.create(
  file=open("balanced_training_set.jsonl", "rb"),
  purpose="fine-tune"
)

In [9]:
response

FileObject(id='file-rGRoV2KwDBuS5IWnM7FkTNUX', bytes=1405172, created_at=1731106374, filename='balanced_training_set.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [10]:
training_file_id = response.id
training_file_id

'file-rGRoV2KwDBuS5IWnM7FkTNUX'

In [11]:
response = client.files.create(
  file=open("balanced_validation_set.jsonl", "rb"),
  purpose="fine-tune"
)

In [12]:
response

FileObject(id='file-939WB6YS4Fw4mhxJe5MnXE0Z', bytes=344491, created_at=1731106374, filename='balanced_validation_set.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [13]:
validation_file_id = response.id
validation_file_id

'file-939WB6YS4Fw4mhxJe5MnXE0Z'

In [14]:
start = time.time()

response = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  validation_file=validation_file_id,
  model=our_model
)

In [15]:
fine_tuning_job_id = response.id
fine_tuning_job_id

'ftjob-w9DSpiWh85sGbpzySBvBejAa'

In [16]:
status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
status

FineTuningJob(id='ftjob-w9DSpiWh85sGbpzySBvBejAa', created_at=1731106376, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=[], seed=23050200, status='validating_files', trained_tokens=None, training_file='file-rGRoV2KwDBuS5IWnM7FkTNUX', validation_file='file-939WB6YS4Fw4mhxJe5MnXE0Z', estimated_finish=None, integrations=[], user_provided_suffix=None)

## Run the Finetuned Model

Check whether it ends in your [fine-tuning UI](https://platform.openai.com/finetune/).

In [17]:
status_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id).status
while status_status not in ["succeeded", "failed"]:
  time.sleep(2)
  status_status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id).status

In [18]:
end = time.time()
print(f"{end - start:.4f} seconds")

3017.9023 seconds


In [19]:
status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
status

FineTuningJob(id='ftjob-w9DSpiWh85sGbpzySBvBejAa', created_at=1731106376, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::ARSv3SWc', finished_at=1731109383, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-7xfU7MhphvesdkdmpMmd5CeS'], seed=23050200, status='succeeded', trained_tokens=937668, training_file='file-rGRoV2KwDBuS5IWnM7FkTNUX', validation_file='file-939WB6YS4Fw4mhxJe5MnXE0Z', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [20]:
fine_tuned_model = status.fine_tuned_model
fine_tuned_model

'ft:gpt-4o-mini-2024-07-18:personal::ARSv3SWc'

In [21]:
y_pred = []
start = time.time()

for i in range(len(test_set)):
  completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=test_set[i]
  )
  y_pred.append(str_to_bool(completion.choices[0].message.content))

end = time.time()
print(f"{end - start:.4f} seconds")

192.3659 seconds


## Evaluation

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

f1_score = f1_score(y_test, y_pred)
print(f"F1 Score: {f1_score}")

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

[[ 56   9]
 [ 11 118]]
Accuracy: 0.8969072164948454
F1 Score: 0.921875
Recall: 0.9147286821705426
